In [ ]:
from drizzlepac.astrodrizzle import AstroDrizzle as asdriz
import os,glob

In [ ]:
filtername = 'F336W'
epoch = '20200529'
rootnamelist = ['ie4502z6q',
                'ie4502z8q',
                'iea4502zaq',
               ]
filesuffix = 'flc.fits'
filepath = '/Users/kbhirombhakdi/_work/_data/15974/MAST_2021-05-24T1044/HST/'
outputdestination = '../ASSETS/DRC_{0}/'.format(filtername)
filelist = []
for i in rootnamelist:
    t = filepath + '{0}/{0}_{1}'.format(i,filesuffix)
    filelist.append(t)

In [ ]:
# copy to local and update pointer
t = glob.glob('*')
if 'tmp' in t:
    os.system('rm -R tmp')
os.system('mkdir tmp')
for ii,i in enumerate(filelist):
    t = './tmp/{0}'.format(filelist[ii].split('/')[-1])
    os.system('cp {0} {1}'.format(filelist[ii],t))
    filelist[ii] = t
filelist

In [ ]:
import subprocess
os.environ['CRDS_SERVER_URL'] = 'https://hst-crds.stsci.edu'
os.environ['CRDS_PATH'] = os.path.abspath(os.path.join('.', 'reference_files'))
for i in filelist:
    subprocess.check_output('crds bestrefs --files {0} --sync-references=1 --update-bestrefs'.format(i), shell=True, stderr=subprocess.DEVNULL)
#####
os.environ['iref'] = os.path.abspath(os.path.join('.', 'reference_files', 'references', 'hst', 'wfc3')) + os.path.sep
!echo $iref

In [ ]:
params = {'build':True,
          'restore':False,
          'overwrite':False,
          'clean':True,
#           'resetbits':512+4096,
          # Step 1 Statis Mask
          'static':True,
          # Step 2 Sky Sub
          'skysub':True,
#           'skymethod':'localmin',
#           'skystat':'median',
          # Step 3 Driz Sep
          'driz_separate':True,
#           'driz_sep_kernel':'turbo',
          'driz_sep_pixfrac':1., # Set this high for CR
          'driz_sep_scale':0.02, # Smaller than native size, better for CR
          # Step 4 Median
          'median':True,
          'combine_type':'mean',
          'combine_nhigh':1, # Reject highest values before computing median, good for CR
          # Step 5 Blot
#           'blot':True,
#           'blot_interp':'poly5',
#           'blot_addsky':True,
          # Step 6 CR
          'driz_cr':True,
#           'driz_cr_corr':True,
#           'driz_cr_ctegrow':0,
          # Step 7 Final Combine
          'driz_combine':True,
          'final_rot':0.,
          'final_wht_type':None,
          'final_refimage':None,
         }

In [ ]:
drizzle_id = 'IVM'
output_prefix = '{0}_{1}_{2}'.format(filtername,epoch,drizzle_id)
##### Drizzle parameters
params['final_wht_type'] = drizzle_id
asdriz(filelist,output=output_prefix,**params)

In [ ]:
# Move files to ASSETS
t = ['{0}_{1}_IVM_drc.fits'.format(filtername,epoch),
     '{0}_{1}_EXP_drc.fits'.format(filtername,epoch),
    ]
for i in t:
    os.system('mv {0} {1}'.format(i,outputdestination))

# For other exposures, we used F336W drc image as the 'final_refimage'.

In [ ]:
final_refimage = '../ASSETS/DRC_F336W/F336W_20200529_IVM_drc.fits'
reftxt = 'refimage20200529'
drizzle_id = 'IVM'
output_prefix = '{0}_{1}_{2}_{3}'.format(filtername,epoch,drizzle_id,reftxt)
##### Drizzle parameters
params['final_refimage'] = final_refimage
params['final_wht_type'] = drizzle_id
asdriz(filelist,output=output_prefix,**params)

In [ ]:
final_refimage = '../ASSETS/DRC_F336W/F336W_20200529_IVM_drc.fits'
reftxt = 'refimage20200529'
drizzle_id = 'EXP'
output_prefix = '{0}_{1}_{2}_{3}'.format(filtername,epoch,drizzle_id,reftxt)
##### Drizzle parameters
params['final_refimage'] = final_refimage
params['final_wht_type'] = drizzle_id
asdriz(filelist,output=output_prefix,**params)